In [ ]:
require 'image'

In [ ]:
require 'torch'

In [ ]:
require 'nn'

In [ ]:
num_examples = 42000
num_classes = 10

In [ ]:
X = torch.Tensor(num_examples,1,28,28)
Y = torch.zeros(num_examples)

In [ ]:
local csvFile = io.open('train.csv', 'r')  
local header = csvFile:read()
local i = 0  
for line in csvFile:lines('*l') do  
  local l = line:split(',')
    i = i + 1;
    if i % 1000 == 0 then
        print(i)
    end
  for key, val in ipairs(l) do
        if key == 1 then
            Y[i] = tonumber(val)+1
        else
            key = key -2
            X[i][1][tonumber(key/28)+1][tonumber(key%28)+1]=val
        end
  end
end
csvFile:close()

In [ ]:
function split(split_input, sep)
    if sep == nil then
        sep = "%s"
    end
    local t = {}; local i = 1
    if split_input == nil then
    	return t
    end
    for str in string.gmatch(split_input, "([^"..sep.."]+)") do
        t[i] = str; i = i + 1
    end
    return t
end

In [ ]:
while true do
    s = file:read()
    if s==nil then
        break
    end
    s = split(s,',')
    Y[tonumber(s[1])] = class_dict[s[2]]
end
    

In [ ]:
print(Y[6])

In [ ]:
conv_net = nn.Sequential()

In [ ]:
conv_net:add(nn.SpatialConvolution(1,32,3,3,1,1,1,1))
conv_net:add(nn.ReLU())
conv_net:add(nn.SpatialMaxPooling(2,2))
conv_net:add(nn.SpatialConvolution(32,64,3,3,1,1,1,1))
conv_net:add(nn.ReLU())
conv_net:add(nn.SpatialMaxPooling(2,2))
conv_net:add(nn.View(64*7*7))
conv_net:add(nn.Linear(64*7*7,625))
conv_net:add(nn.Sigmoid())
conv_net:add(nn.Linear(625,10))
conv_net:add(nn.SoftMax())

In [ ]:
conv_net = torch.load('conv_net_model')

In [ ]:
criterion = nn.CrossEntropyCriterion()
trainer = nn.StochasticGradient(conv_net,criterion)
trainer.learningRate = 0.01
trainer.learningRateDecay = 0.001

In [ ]:
dataset = {}
function dataset:size() return 40000 end
test_data = {}
for i = 1,dataset:size() do
    dataset[i]= {X[i],Y[i]}
end

In [ ]:
print(dataset[39355])

In [ ]:
for iter = 1,1 do
    for i=1,42000 do
        local pred =  conv_net:forward(X[i])
        local loss = criterion:forward(pred,Y[i])
        local gradCriterion = criterion:backward(pred,Y[i])
        conv_net:zeroGradParameters()
        conv_net:backward(X[i],gradCriterion)
        conv_net:updateParameters(0.005)
        if(i%1000==0) then
            print(i)
            print(Y[i])
            t,j = torch.max(pred,1)
            print(j[1])
        end
    end
    torch.save('conv_net_model',conv_net)
    local total = 0.0
    local correct = 0.0
    for i=37001,42000 do
        local pred = conv_net:forward(X[i])
        total = total + 1.0
        t,j = torch.max(pred,1)
        if (j[1]==Y[i]) then
            correct = correct + 1.0
        end
    end
    print('accuracy')
    print(correct/total)
end

In [ ]:
torch.save('conv_net_model',conv_net)

In [ ]:
num_test_examples=28000

In [ ]:
test_example = torch.Tensor(1,28,28)
local csvFile = io.open('test.csv', 'r')  
local header = csvFile:read()
local i = 0  
for line in csvFile:lines('*l') do  
  local l = line:split(',')
    i = i + 1;
    if i % 1000 == 0 then
        print(i)
    end
  for key, val in ipairs(l) do
            key = key -1
            test_example[1][tonumber(key/28)+1][tonumber(key%28)+1]=val
  end
  local pred = conv_net:forward(test_example)
  t,j = torch.max(pred,1)
  print(i..","..j[1]-1)
end
csvFile:close()